CHATGPT Question:

how to easily implement RAG using Hugging face? Let's try to do it as easy as possible. I want to use Python and maybe GitHub for my knowledge base (I have a knowledge object in Markdown format that I will be using to feed the knowledge base)? What I'm trying to do is basically the user ask the pre-trained RAG (easiet Hugging face model out there) about information within my knowledge base and the model then generates an answer using the information I have within the knowledge base. is it possible to do this?

help me with this: how to easily implement RAG using Hugging face? Let's try to do it as easy as possible. I want to use Python and maybe GitHub for my knowledge base (I have a knowledge object in Markdown format that I will be using to feed the knowledge base)? What I'm trying to do is basically the user ask the pre-trained RAG (easiet Hugging face model out there) about information within my knowledge base and the model then generates an answer using the information I have within the knowledge base. is it possible to do this? I am initially testing all this using Colab. I have the following:

In [ ]:
!apt-get install pandoc

In [ ]:
!pip install markdown2

In [ ]:
!pip install transformers datasets

In [ ]:
!pip install transformers datasets pandas

In [5]:
!pip install faiss-cpu

In [ ]:
# !pip install faiss-gpu

In [46]:
import subprocess
import os
import markdown2
import json
import torch
import re
import pandas as pd
import numpy as np
import faiss
from datasets import Dataset

In [9]:
# Converting word document to markdown
def convert_docx_to_md(docx_file, md_file):
    try:
        subprocess.run(['pandoc', '-s', docx_file, '-o', md_file], check=True)
        print(f"Conversion successful: '{docx_file}' to '{md_file}'")
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

path = '/content/Data_Curation_KO.docx'
convert_docx_to_md(path, 'Data_Curation.md')

Conversion successful: '/content/Data_Curation_KO.docx' to 'Data_Curation.md'


In [10]:
def markdown_to_json(markdown_text):
    # Split the markdown text into sections based on the numbered headings
    sections = re.split(r'\n\d+\.\s+', markdown_text)[1:]  # Skip the initial empty split

    data = []
    for section in sections:
        # Extract the heading and the content
        heading, *content = section.strip().split('\n')

        # Further split content into bullet points
        bullet_points = [line.strip('- ').strip() for line in content if line.startswith('-')]

        section_data = {
            'heading': heading.strip(),
            'content': bullet_points
        }
        data.append(section_data)

    return data

In [11]:
with open('/content/Data_Curation.md', 'r') as file:
    markdown_content = file.read()

json_data = markdown_to_json(markdown_content)

# Save the JSON data to a file
json_file = 'Data_Curation.json'
with open(json_file, 'w') as f:
    json.dump(json_data, f, indent=4)

Knowledge Base to Hugging Face Datasets:
- If your knowledge base is small, you might manually create a dataset in Colab. For larger datasets, consider following the Hugging Face documentation to upload your dataset to their platform.

- If manual creation is feasible, convert your data into a format that the RAG model can use, such as a list of dictionaries, each containing a piece of information.

# Approach 1: Dummy Dataset

In [ ]:
#!pip install transformers datasets pandas

In [ ]:
! pip install --upgrade torch transformers

In [7]:
from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration, DPRQuestionEncoder

# Load a pre-trained RAG model
tokenizer = RagTokenizer.from_pretrained("facebook/rag-token-base")
question_encoder = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
retriever = RagRetriever.from_pretrained("facebook/rag-token-base", index_name="exact", use_dummy_dataset=True)
generator = RagSequenceForGeneration.from_pretrained("facebook/rag-token-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.55k [00:00<?, ?B/s]

(…)_encoder_tokenizer/tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

question_encoder_tokenizer/vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

(…)ncoder_tokenizer/special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for wiki_dpr contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wiki_dpr
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

  0%|          | 0/10 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/2.06G [00:00<?, ?B/s]

In [21]:
from transformers import DPRQuestionEncoder
question_encoder = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base")

config.json:   0%|          | 0.00/493 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [40]:
# Define your question
question = "What is Data Curation?"

# Encode the question
question_encoding = tokenizer(question, return_tensors="pt", padding=True, truncation=True)
input_ids = question_encoding["input_ids"]
attention_mask = question_encoding["attention_mask"]

# Use the DPRQuestionEncoder to encode the question for the retriever
with torch.no_grad():
    question_hidden_states = question_encoder(input_ids).pooler_output

# Convert PyTorch tensors to NumPy arrays for the retriever
question_hidden_states_np = question_hidden_states.cpu().numpy()
input_ids_np = input_ids.cpu().numpy()

# Perform retrieval
with torch.no_grad():
    retrieval_results = retriever(question_hidden_states=question_hidden_states_np, question_input_ids=input_ids_np)

# Context input ids and attention mask for the generator
context_input_ids = retrieval_results["context_input_ids"]
context_attention_mask = retrieval_results["context_attention_mask"]

# Number of documents to be retrieved
n_docs = 5

# Flatten context_input_ids and context_attention_mask to match the input_ids
context_input_ids = torch.tensor(context_input_ids).view(1, -1)[:,:n_docs*input_ids.size(1)]
context_attention_mask = torch.tensor(context_attention_mask).view(1, -1)[:,:n_docs*input_ids.size(1)]

# Placeholder doc_scores (size [batch_size, n_docs])
doc_scores = torch.ones((input_ids.shape[0], n_docs))

# Generate an answer using the generator
with torch.no_grad():
    generated = generator(input_ids=input_ids,
                          attention_mask=attention_mask,
                          context_input_ids=context_input_ids,
                          context_attention_mask=context_attention_mask,
                          doc_scores=doc_scores,
                          num_return_sequences=1)

# Inspect the output from the generator
print("Generated output type:", type(generated))
print("Generated output:", generated)

# Check if generated output is a tensor and decode it
if isinstance(generated, torch.Tensor):
    answer = tokenizer.decode(generated[0], skip_special_tokens=True)
    print("Answer:", answer)
else:
    print("Unexpected output format from the generator")


Generated output type: <class 'transformers.models.rag.modeling_rag.RetrievAugLMMarginOutput'>
Generated output: RetrievAugLMMarginOutput(loss=None, logits=tensor([[[ 13.6258,  -1.7973,   6.9678,  ...,  -3.3383,  -3.8260,   5.4638],
         [ 13.6258,  -1.7973,   6.9678,  ...,  -3.3383,  -3.8260,   5.4638],
         [-17.4461,  -3.9540,   2.9546,  ...,  -3.8370,  -3.6517,  -0.7094],
         ...,
         [-24.1339,  -4.3240,   5.6375,  ...,   0.2479,  -1.3351,   2.1525],
         [-16.6575,  -3.5976,   6.8172,  ...,  -1.8138,  -4.3158,   0.8610],
         [-23.0490,  -3.7092,   9.4855,  ...,  -0.7204,   0.1401,   7.4595]]]), doc_scores=tensor([[1., 1., 1., 1., 1.]]), past_key_values=((tensor([[[[ 2.7406e-01,  4.6150e-01,  1.9901e+00,  ...,  9.9268e-01,
            2.8700e+00, -1.3778e+00],
          [-7.4523e-02,  6.4510e-01,  1.7084e+00,  ...,  9.1152e-01,
            2.5264e+00, -1.6802e+00],
          [-1.1017e+00, -7.8395e-01, -2.0451e+00,  ..., -2.8963e+00,
           -3.9539e+0

The RAG (Retriever-Augmented Generation) model, as implemented in the Hugging Face Transformers library, is typically configured to retrieve information from a large-scale dataset (like Wikipedia) to assist in generating answers.

In [41]:
# Define your question
question = "What is Data Curation?"

# Encode the question
question_encoding = tokenizer(question, return_tensors="pt", padding=True, truncation=True)
input_ids = question_encoding["input_ids"]
attention_mask = question_encoding["attention_mask"]

# Use the DPRQuestionEncoder to encode the question for the retriever
with torch.no_grad():
    question_hidden_states = question_encoder(input_ids).pooler_output

# Convert PyTorch tensors to NumPy arrays for the retriever
question_hidden_states_np = question_hidden_states.cpu().numpy()
input_ids_np = input_ids.cpu().numpy()

# Perform retrieval
with torch.no_grad():
    retrieval_results = retriever(question_hidden_states=question_hidden_states_np, question_input_ids=input_ids_np)

# Context input ids and attention mask for the generator
context_input_ids = retrieval_results["context_input_ids"]
context_attention_mask = retrieval_results["context_attention_mask"]

# Number of documents to be retrieved
n_docs = 5

# Flatten context_input_ids and context_attention_mask to match the input_ids
context_input_ids = torch.tensor(context_input_ids).view(1, -1)[:,:n_docs*input_ids.size(1)]
context_attention_mask = torch.tensor(context_attention_mask).view(1, -1)[:,:n_docs*input_ids.size(1)]

# Placeholder doc_scores (size [batch_size, n_docs])
doc_scores = torch.ones((input_ids.shape[0], n_docs))

with torch.no_grad():
    generated_output = generator(input_ids=input_ids,
                                 attention_mask=attention_mask,
                                 context_input_ids=context_input_ids,
                                 context_attention_mask=context_attention_mask,
                                 doc_scores=doc_scores,
                                 num_return_sequences=1)

# Extract the logits
logits = generated_output.logits

# Extract the generated token IDs from the logits
# Assuming you want the first generated sequence
generated_token_ids = torch.argmax(logits, dim=-1)[0]

# Decode the generated token IDs
answer = tokenizer.decode(generated_token_ids, skip_special_tokens=True)
print("Answer:", answer)

Answer: (( of variance / size is more prone to Type II error). The terminology of ANOVA is largely from the statistical design of experiments. The experimenter adjusts factors and measures responses in an attempt to


# RAG approach 2: FIASS

FAISS

FAISS, developed by Facebook AI, is a library for efficient similarity search and clustering of dense vectors. It is primarily used for quickly searching for items in a large dataset that are similar to a query item. Key features and use cases of FAISS include:

- Vector Search: Efficiently searches through large collections of high-dimensional vectors (e.g., embeddings) for nearest neighbors.

- Clustering: Can be used to cluster large sets of vectors, which is helpful in organizing or summarizing data.

- Performance: Optimized for speed and scalability, especially on datasets with millions to billions of vectors.

- Applications: Commonly used in information retrieval, computer vision, and large-scale data analysis where quick vector similarity calculations are needed.


RAG

RAG, developed by Hugging Face, combines a retriever (like DPR) with a generator (like BART or T5) and is used for tasks like question answering and text generation. Key aspects of RAG include:

- Retrieval-Augmented Generation: Uses a retriever model to fetch relevant context from a large corpus (like Wikipedia) and a generator model to produce an answer or generate text based on that context.

- NLP Tasks: Primarily used for complex natural language processing tasks where context from external documents enhances the quality of the generated text.
Transformer Models: Integrates transformer-based models for both the retriever and generator parts, leveraging their capabilities in understanding and generating human-like text.

- Applications: Suitable for question answering, chatbots, and any application where generating informed and contextually relevant text is important.


Key Differences
- Purpose: FAISS is focused on the fast retrieval of similar items based on vector similarity. RAG is a more complex system for generating text based on retrieved information.

- Functionality: FAISS is a library for similarity search and clustering, while RAG is an end-to-end model combining retrieval and text generation.

- Use Cases: FAISS is used in scenarios where you need to find the most similar items in a large dataset, such as for recommendations or similarity-based searches. RAG is used in scenarios where you need to generate text that is informed by and contingent on large external corpora.

In summary, FAISS is specialized for similarity search in large datasets, whereas RAG is designed for generating text based on information retrieved from large corpora. They serve different roles in the machine learning and NLP ecosystem.

In [ ]:
!pip install -U sentence-transformers

In [ ]:
# Load your data
with open('Data_Curation.json', 'r') as file:
    data = json.load(file)

In [ ]:
#from sentence_transformers import SentenceTransformer

# Choose a model for embeddings (e.g., sentence transformers)
#model = SentenceTransformer('all-MiniLM-L6-v2')

# Process your data to get embeddings
# Assuming each item in your JSON is a document
#embeddings = [model.encode(doc['content']) for doc in data]

# Convert embeddings to FAISS compatible format
#d = len(embeddings[0])  # dimension of embeddings
#index = faiss.IndexFlatL2(d)
#index.add(np.array(embeddings).astype('float32'))  # add vectors to the index

In [48]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

embeddings = []
for doc in data:
    # Assuming 'doc' is a dictionary with a key 'content' that contains the text
    embedding = model.encode(doc['content'])
    embeddings.append(embedding)
    print(f"Document: {doc['content']}, Embedding Shape: {embedding.shape}")

# Check if all embeddings have the same shape
embedding_lengths = set(len(embedding) for embedding in embeddings)
print("Unique embedding lengths:", embedding_lengths)

Document: ['Curation means "to take care". Like a librarian\\\'s role, data', '**Importance**: Ensures data used for machine learning and deep'], Embedding Shape: (2, 384)
Document: ['**Early Days (1950s-1980s)**: Focus on developing algorithms with', '**Rise of Large-Scale Datasets (1990s)**: Increased computing power', '**The 2000s -- 2010s**: Emergence of automated tools for data'], Embedding Shape: (3, 384)
Document: ['Data collection: Gathering data from various sources.', 'Data cleaning: Eliminating duplicates, handling outliers, etc.', 'Data annotation: Labeling data for supervised learning.', 'Data transformation: Converting data into a suitable format for', 'Data integration: Combining data from multiple sources.', 'Data maintenance: Updating or augmenting data over time.', 'Data Augmentation: Techniques like rotation, flipping to improve', 'Data Storage and Management: Using SQL, NoSQL databases, and data'], Embedding Shape: (8, 384)
Document: ['Healthcare: Managing patient r

In [49]:
model = SentenceTransformer('all-MiniLM-L6-v2')

averaged_embeddings = []
for doc in data:
    # Compute embeddings for each bullet point/sentence
    embeddings = model.encode(doc['content'], convert_to_tensor=True)

    # Average the embeddings to get a single vector per document
    avg_embedding = torch.mean(embeddings, dim=0)
    averaged_embeddings.append(avg_embedding.cpu().numpy())

# Convert list of averaged embeddings to a 2D numpy array
embeddings_array = np.vstack(averaged_embeddings)

# Check if all embeddings now have the same shape
print("Shape of each embedding:", embeddings_array.shape[1])

Shape of each embedding: 384


In [50]:
d = embeddings_array.shape[1]  # Dimension of embeddings
index = faiss.IndexFlatL2(d)

# Add vectors to the index
index.add(embeddings_array.astype('float32'))

In [51]:
def retrieve_documents(query, k=5):
    # Encode the query to the same embedding space
    query_embedding = model.encode(query, convert_to_tensor=True)
    query_embedding = query_embedding.cpu().numpy()

    # Search in the index
    distances, indices = index.search(np.array([query_embedding]), k)

    # Retrieve and return the documents
    return [data[i] for i in indices[0]]

However, integrating a custom retriever directly with RAG is an advanced task that requires a deep understanding of the RAG architecture and the Transformers library. Due to its complexity, this task might be beyond the scope of typical use cases and may require specialized knowledge in machine learning and natural language processing.

For many applications, a simpler approach might be to use the custom FAISS retriever independently (as shown in the document retrieval example) and then feed the retrieved documents into a separate generative model as needed.

In [52]:
# Example query
query = "What is Data Curation?"
retrieved_docs = retrieve_documents(query)
for doc in retrieved_docs:
    print(doc)

{'heading': 'Conclusion:', 'content': ['Data curation is crucial in Machine Learning and Deep Learning,', 'Importance of Data Curation: Essential for high-quality, relevant', 'Data Curation Steps: collection, cleaning, annotation,']}
{'heading': 'Data Curation in Machine Learning and Deep Learning:', 'content': ['Curation means "to take care". Like a librarian\\\'s role, data', '**Importance**: Ensures data used for machine learning and deep']}
{'heading': 'Future Trends in Data Curation:', 'content': ['Artificial Intelligence and Automation: Streamlining data curation', 'Open Data Initiatives: Emphasizing transparency and collaboration.', 'Advanced Algorithms: For bias detection and data privacy.']}
{'heading': 'Open-Source Data Curation Platform: Lightly.AI: Optimizing Image', 'content': ['Focus: Supervised learning for image datasets.', 'Open Source: Operates under the MIT license, focusing on', 'Self-Supervised Learning and Active Learning: Analyzes data', 'Data Analytics Features:

In [54]:
# Example query 2
query = "What are some Open-Source Data Curation Platforms?"
retrieved_docs = retrieve_documents(query)
for doc in retrieved_docs:
    print(doc)

{'heading': 'Conclusion:', 'content': ['Data curation is crucial in Machine Learning and Deep Learning,', 'Importance of Data Curation: Essential for high-quality, relevant', 'Data Curation Steps: collection, cleaning, annotation,']}
{'heading': 'Future Trends in Data Curation:', 'content': ['Artificial Intelligence and Automation: Streamlining data curation', 'Open Data Initiatives: Emphasizing transparency and collaboration.', 'Advanced Algorithms: For bias detection and data privacy.']}
{'heading': 'Open-Source Data Curation Platform: Lightly.AI: Optimizing Image', 'content': ['Focus: Supervised learning for image datasets.', 'Open Source: Operates under the MIT license, focusing on', 'Self-Supervised Learning and Active Learning: Analyzes data', 'Data Analytics Features: Provides insights into collected data with', 'Visual Quality Analysis: Shows samples that were kept and removed.', 'Data Selection Automation: Selects the best 1% from large datasets', 'Pipeline Management: Manages

# Approaxh 3: FIASS + RAG model

How FAISS and LLMs Can Be Used Together:

- Embedding Retrieval: LLMs can be used to convert text into high-dimensional embeddings (vectors). FAISS can then index these embeddings, enabling fast and efficient similarity searches. For instance, you might use an LLM to embed a collection of documents and then use FAISS to quickly find the most similar documents to a given query embedding.

- Augmenting Language Models: In some advanced applications, FAISS can be used as part of a retrieval-augmented pipeline where a language model first generates an embedding for a query, and then FAISS retrieves relevant context or information, which can then be used to enhance the language model's output.

Custom Retriever Implementation: You would need to implement a custom retriever class that interfaces with the FAISS index you created. This retriever should be able to take a query, find the most relevant documents from your FAISS index, and return them in a format compatible with the RAG model.

Modify RAG to Use the Custom Retriever: The next step would be to modify the RAG model to use your custom retriever instead of its default retriever. This involves understanding and potentially altering the internal workings of the RAG model in the Transformers library, which can be quite complex.

Using RAG for Generation: Once the RAG model is set up with your custom retriever, you can use it to generate answers or content based on the retrieved documents.

In [ ]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

class CustomFaissRetriever:
    def __init__(self, index, data, model_name='all-MiniLM-L6-v2'):
        self.index = index
        self.data = data
        self.model = SentenceTransformer(model_name)

    def retrieve(self, query, k=5):
        query_embedding = self.model.encode(query)
        query_embedding = np.array(query_embedding).astype('float32').reshape(1, -1)
        distances, indices = self.index.search(query_embedding, k)
        return [(self.data[i], distances[0][j]) for j, i in enumerate(indices[0])]